# Consigna

Link a la consigna: https://piazza.com/class_profile/get_resource/jkr2voxi1yw4wt/jkr2vqrpsea4xf

A partir de la plataforma online (e-shop) de los países en los que opera, Nintendo tiene información de ventas de videojuegos diarias digitales por país en el siguiente RDD: (id_videojuego, codigo_pais, fecha, visitas_diarias, total_ventas_diarias). 

Por otro lado se tienen otro RDD que tiene información de todos los videojuegos que se venden en su plataforma con el siguiente formato (id_videojuego, titulo, rating_pegi, rating_esbr). Tener en cuenta que un mismo videojuego se puede vender en distintos países y esos nos permitirá obtener métricas a nivel global. 

Con esta información escribir un programa en pySpark que permita: 

a) Obtener el videojuego con más ventas digitales globales (es decir en todos los países) en un RDD con el siguiente formato: (id_videojuego, titulo, total), siendo total la cantidad total de ventas digitales globales

b) Para el videojuego con mas ventas, obtener cual es el país para el cual ser registra una mayor tasa de conversión (es decir, mayor total_ventas_diarias / visitas_diarias)

In [1]:
from pyspark import SparkContext

sc = SparkContext.getOrCreate()

In [2]:
datos_ventas_diarias = [
    (1, 'AR', '2018-01-01', 30, 10),
    (1, 'ES', '2018-01-01', 23, 13),
    (2, 'US', '2018-01-04', 45, 5),
    (2, 'MX', '2018-01-04', 20, 10),
    (2, 'US', '2018-01-06', 50, 15),
    (3, 'AR', '2018-01-06', 10, 2),
    (1, 'US', '2018-01-06', 14, 4),
    (3, 'ES', '2018-01-10', 34, 11),
    (4, 'ES', '2018-01-11', 42, 24),
    (4, 'US', '2018-01-11', 83, 34),
    (4, 'AR', '2018-01-11', 27, 20),
    (4, 'MX', '2018-01-11', 47, 18),
    (4, 'AR', '2018-01-20', 10, 0),
    (4, 'US', '2018-01-21', 34, 2),
    (4, 'ES', '2018-01-21', 25, 7)
]

datos_videojuegos = [
    (1, 'Zelda: Breath of the Wild', 9, 8),
    (2, 'Mario Kart', 9, 7),
    (3, 'Splatoon 2', 11, 8),
    (4, 'Monster Hunter Generations Ultimate', 13, 10)
]

## Punto A

Sean:

* rdd_vd = rdd con los datos de ventas diarias de cada id_videojuego.
* rdd_vg = rdd con el título y rating asociado a cada id_videojuego.

In [3]:
rdd_vd = sc.parallelize(datos_ventas_diarias)
rdd_vg = sc.parallelize(datos_videojuegos)

### Versión realizando el join al principio.

In [4]:
#  No aplico map a rdd_vg porque el join va a tomar el id_videojuego como clave, y titulo como valor.
#  En caso de ser necesario, seria rdd_vg.map(lambda x: (x[0], x[1]))
rdd_unido = rdd_vd.map(lambda x: (x[0], x[4])).join(rdd_vg)

#  x[1][1] == titulo
#  x[1][0] == total_ventas_diarias
#  Registros con pinta ((id_videojuego, titulo), total_ventas_diarias)
rdd_unido = rdd_unido.map(lambda x: ((x[0], x[1][1]), x[1][0]))

#  Hacemos reduceByKey para obtener el total de ventas por videojuego, y luego reduce nos devuelve el mas vendido.
juego_mas_vendido = rdd_unido.reduceByKey(lambda a,b: a+b).reduce(lambda a,b: a if a[1] > b[1] else b)

#  Armamos un RDD con la tupla obtenida y aplicamos map para obtener el formato pedido en la consigna.
rdd_resultado_a = sc.parallelize([juego_mas_vendido]).map(lambda x: (x[0][0], x[0][1], x[1]))
rdd_resultado_a.collect()

[(4, 'Monster Hunter Generations Ultimate', 105)]

### Versión realizando el join al final.

En los criterios de corrección, se menciona que esta versión es la ideal.

In [5]:
rdd_a = rdd_vd.map(lambda x: (x[0], x[4]))
rdd_a = rdd_a.reduceByKey(lambda a,b: a+b).reduce(lambda a,b: a if a[1] > b[1] else b)
rdd_a = sc.parallelize([rdd_a])
rdd_a = rdd_a.join(rdd_vg.map(lambda x: (x[0], x[1]))).map(lambda x: (x[0], x[1][1], x[1][0]))
rdd_a.collect()

[(4, 'Monster Hunter Generations Ultimate', 105)]

## Punto B

In [6]:
#  Me quedo como valor tuplas de la pinta (codigo_pais, total_vistas_diarias, total_ventas_diarias)
#  Hago un join con el resultado del punto A para quedarme con los registros del juego mas vendido obtenido en A.
rdd_b = rdd_vd.map(lambda x: (x[0], (x[1], x[3], x[4]))).join(rdd_resultado_a)

#  Luego aplico map para pasar el codigo de pais como clave.
rdd_b = rdd_b.map(lambda x: (x[1][0][0], (x[1][0][1], x[1][0][2])))

In [7]:
rdd_b.collect()

[('ES', (42, 24)),
 ('US', (83, 34)),
 ('AR', (27, 20)),
 ('MX', (47, 18)),
 ('AR', (10, 0)),
 ('US', (34, 2)),
 ('ES', (25, 7))]

In [8]:
#  Ahora reduceByKey para obtener los totales de ventas y vistas para cada pais.
rdd_b = rdd_b.reduceByKey(lambda a,b: (a[0]+b[0], a[1]+b[1]))
rdd_b.collect()

[('US', (117, 36)), ('ES', (67, 31)), ('AR', (37, 20)), ('MX', (47, 18))]

In [9]:
#  Luego map para obtener la tasa de conversion.
rdd_b = rdd_b.map(lambda x: (x[0], float(x[1][1]) / x[1][0]))
rdd_b.collect()

[('US', 0.3076923076923077),
 ('ES', 0.4626865671641791),
 ('AR', 0.5405405405405406),
 ('MX', 0.3829787234042553)]

In [10]:
#  Finalmente aplicamos reduce para obtener la maxima de las tasas de conversion.
resultado_b = rdd_b.reduce(lambda a,b: a if a[1] > b[1] else b)
resultado_b

('AR', 0.5405405405405406)